**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
ratings.groupby('userId').agg({'movieId':'count'})

,movieId
userId,
1,232
2,29
3,39
4,216
5,44
...,...
606,1115
607,187
608,831


In [5]:
ratings_100=ratings.groupby('userId').agg({'movieId':'count'})
ratings_100=ratings_100.loc[ratings_100['movieId']>=100]
ratings_100

,movieId
userId,
1,232
4,216
6,314
7,152
10,140
...,...
605,221
606,1115
607,187


In [6]:
ratings_new=ratings.merge(ratings_100, how='inner',on=['userId'])
ratings_new

,userId,movieId_x,rating,timestamp,movieId_y
0,1,1,4.0,964982703,232
1,1,3,4.0,964981247,232
2,1,6,4.0,964982224,232
3,1,47,5.0,964983815,232
4,1,50,5.0,964982931,232
...,...,...,...,...,...
84308,610,166534,4.0,1493848402,1302
84309,610,168248,5.0,1493850091,1302
84310,610,168250,5.0,1494273047,1302
84311,610,168252,5.0,1493846352,1302


In [7]:
ratings_new_2=ratings_new.groupby('userId').agg({'timestamp': lambda x: x.max() - x.min()})

In [8]:
ratings_new_2.mean().round(1)/(3600*24)

timestamp    511.461597
dtype: float64

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [10]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
new_base=client_base.drop(columns='address')
new_base=new_base.merge(rzd, how = 'left', on = 'client_id')
new_base=new_base.merge(auto, how = 'left', on = 'client_id')
new_base=new_base.merge(air, how = 'left', on = 'client_id')
new_base

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [16]:
new_base_2=client_base.merge(rzd, how = 'left', on = 'client_id')
new_base_2=new_base_2.merge(auto, how = 'left', on = 'client_id')
new_base_2=new_base_2.merge(air, how = 'left', on = 'client_id')
new_base_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0



**Задание 3**

3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

для каждого вызова приложения я бы добавлял столбцы user_id,широта, долгота, date_time и порядковый номер строки, тогда мы сможем связать факт покупки/визита с наиболее близким по времени (слева) местоположением клиента. 
можно определить координаты мест, областей а также различных типов геообъектов из какой нибудь доступной базы.
можно при вызове приложения (при условии достаточного быстродействия базы) сравнивать близость клиента к определенным, важным для нас точкам